In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
image_dir = '/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/'
base_dir = '/kaggle/input/ocular-disease-recognition-odir5k/'

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from utils import extract_target_array

In [ ]:
data_info = pd.read_csv(base_dir+'full_df.csv')
targets = data_info['target']

In [ ]:
myopia = data_info[data_info['M']==1]
normal = data_info[data_info['N']==1]

In [ ]:
normal=normal.sample(frac=0.15,random_state=200) #random state is a seed value
normal

In [ ]:
data_info = pd.concat([myopia,normal])

In [ ]:
train_df=data_info.sample(frac=0.90,random_state=200) #random state is a seed value
test_df=data_info.drop(train_df.index)

In [ ]:
def extract_target(target):
    target = target.replace('[','').replace(']','')
    target = target.split(',')
    target = [int(t.strip()) for t in target]
    return np.argmax(np.array(target))
    return target

In [ ]:
def load_images(image_fnames,images_dir):
    images = []
    n = 0
    for fname in image_fnames:
        image = cv2.imread(images_dir+fname)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        images.append(image)
        print(n); n+=1
        print(image.shape)
        if n > 500:
            break
    return np.array(images)

In [ ]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
    array = tf.io.serialize_tensor(array)
    return array

In [ ]:
def parse_single_image(image, label):
  #define the dictionary -- the structure -- of our single example
    data = {
            'height' : _int64_feature(image.shape[0]),
            'width' : _int64_feature(image.shape[1]),
            'depth' : _int64_feature(image.shape[2]),
            'raw_image' : _bytes_feature(serialize_array(image)),
            'label' : _int64_feature(label)
        }
   #create an Example, wrapping the single features
    output = tf.train.Example(features=tf.train.Features(feature=data))
    return output

In [ ]:
def parse_tfr_element(element):
  #use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'label':tf.io.FixedLenFeature([], tf.int64),
      'raw_image' : tf.io.FixedLenFeature([], tf.string),
      'depth':tf.io.FixedLenFeature([], tf.int64),
    }
    content = tf.io.parse_single_example(element, data)
  
    height = content['height']
    width = content['width']
    depth = content['depth']
    label = content['label']
    raw_image = content['raw_image']
    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(raw_image, out_type=tf.uint8)
    feature = tf.reshape(feature, shape=[height,width,depth])
    feature = tf.image.resize(feature, [256, 256])/255
    return (feature, label)

In [ ]:
def images2tfrecord(df,filename):
    #
    filename= filename+".tfrecords"
    writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our data to disk
    count = 0
    for idx,row in df.iterrows():
        image = cv2.imread(image_dir+row['filename'])
        current_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        current_label = row['N']
        #current_label = extract_target(current_label)
        out = parse_single_image(image=current_image, label=current_label)
        writer.write(out.SerializeToString())
        count += 1
    writer.close()
    print(f"Wrote {count} elements to TFRecord")
    return count 

In [ ]:
images2tfrecord(train_df,'train_data_record')
images2tfrecord(test_df,'test_data_record')

In [ ]:
raw_image_dataset_train = tf.data.TFRecordDataset('train_data_record.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset_train = raw_image_dataset_train.map(_parse_image_function)
raw_image_dataset_test = tf.data.TFRecordDataset('test_data_record.tfrecords')
parsed_image_dataset_test = raw_image_dataset_test.map(_parse_image_function)

In [ ]:
def get_dataset(filename):
  #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

  #pass every single feature through our mapping function
    dataset = dataset.map(
      parse_tfr_element
  ) 
    return dataset

In [ ]:
map_dataset_train = get_dataset('train_data_record.tfrecords')
map_dataset_test = get_dataset('test_data_record.tfrecords')

In [ ]:
for image_features in map_dataset_train:
    image_raw = image_features[0].numpy()
    label = image_features[1]
    print(label)
    plt.imshow(image_raw)
    break
    # to iterate over the dataset

In [ ]:
batched_dataset_train = map_dataset_train.batch(32, drop_remainder=True)
batched_dataset_test = map_dataset_test.batch(32, drop_remainder=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D,Flatten, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy, CategoricalCrossentropy, binary_crossentropy

In [ ]:
base_model = tf.keras.applications.VGG19(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
inputs = tf.keras.Input(shape=(256, 256, 3))
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')
fc_layer = tf.keras.layers.Dense(256,activation='relu')
x = base_model(inputs, training=False)
for layer in base_model.layers:
    layer.trainable = False
    
#x = tf.keras.layers.Dropout(0.4)(x)
x = Flatten()(x)
#x = fc_layer(x)
x = tf.keras.layers.Dropout(0.4)(x)
#outputs = prediction_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer='adam',loss=binary_crossentropy,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(batched_dataset_train,epochs=15,validation_data=batched_dataset_test)

In [ ]:
model.evaluate(batched_dataset_test)

In [ ]:
model.save('/kaggle/working/myopia.h5')